# Pulled QM9 and yukawa interaction from public kernels
https://www.kaggle.com/robikscube/feature-pull-qm9-custom-objective-in-python

https://www.kaggle.com/robikscube/feature-pull-workflow-for-each-type-lb-1-28

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import dask.dataframe as dd

## Note: df3 has more precision than df2

In [4]:
type_dict = {0:'1JHC',
              1:'1JHN',
              2:'2JHC',
              3:'2JHH',
              4:'2JHN',
              5:'3JHC',
              6:'3JHH',
              7:'3JHN'}

In [21]:
df3.head()

,type,molecule_name,id,NO,rc_A,rc_B,rc_C,mu,alpha,homo,...,gap,zpve,Cv,freqs_min,freqs_max,freqs_mean,mulliken_min,mulliken_max,mulliken_atom_0,mulliken_atom_1
0,2,3,4658147,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,...,0.3351,0.026841,8.574,549.7648,3557.8599,1683.122114,-0.207019,0.207019,0.207019,-0.207019
1,0,3,4658148,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,...,0.3351,0.026841,8.574,549.7648,3557.8599,1683.122114,-0.207019,0.207019,0.207019,-0.207019
2,6,3,4658149,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,...,0.3351,0.026841,8.574,549.7648,3557.8599,1683.122114,-0.207019,0.207019,0.207019,0.207019
3,0,3,4658150,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,...,0.3351,0.026841,8.574,549.7648,3557.8599,1683.122114,-0.207019,0.207019,0.207019,-0.207019
4,2,3,4658151,NaN,0.0,35.610036,35.610036,0.0,16.28,-0.2845,...,0.3351,0.026841,8.574,549.7648,3557.8599,1683.122114,-0.207019,0.207019,0.207019,-0.207019


In [22]:
test = pd.read_csv('../input/test.csv')

In [23]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


In [ ]:
for tt in ['train','test']:
    for t in tqdm(range(0,8)):
        df = pd.read_parquet(f'../data/FE018/FE018-{tt}-{type_dict[t]}.parquet')
        df2 = pd.read_parquet(f'../data/FE019/raw/{tt}_{t}_FE_yuk_QM9.parquet')
        df2['type'] = df2['type'].map(type_dict)
        df2 = df2.drop(['rc_A', 'rc_B', 'rc_C', 'mu', 'alpha', 'homo', 'lumo',
               'gap', 'zpve', 'Cv', 'freqs_min', 'freqs_max', 'freqs_mean',
               'mulliken_min', 'mulliken_max', 'mulliken_atom_0', 'mulliken_atom_1'], axis=1)
        df3 = pd.read_parquet(f'../data/FE019/raw/{tt}_public_kernel_QM9.parquet')
        df3 = df3[df3['type'] == t]
        df3['type'] = df3['type'].map(type_dict)
        df3 = df3.drop(['NO','molecule_name'], axis=1)

        df_new = pd.merge(df, df2, on=['id','type'])
        df_new = pd.merge(df_new, df3, on=['id','type'])
        df_new.to_parquet(f'../data/FE019/FE019-{tt}-{type_dict[t]}.parquet')
        if len(df_new) != len(df):
            print('Changed length. Something is wrong')
            raise

 62%|██████▎   | 5/8 [12:18<07:07, 142.34s/it]

In [1]:
# # Using Dask
## ITS SLOWER - AGH!
# for tt in ['train','test']:
#     for t in tqdm(range(0,8)):
#         df = dd.read_parquet(f'../data/FE018/FE018-{tt}-{type_dict[t]}.parquet')
#         df2 = dd.read_parquet(f'../data/FE019/raw/{tt}_{t}_FE_yuk_QM9.parquet')
#         df2['type'] = df2['type'].map(type_dict)
#         df2 = df2.drop(['rc_A', 'rc_B', 'rc_C', 'mu', 'alpha', 'homo', 'lumo',
#                'gap', 'zpve', 'Cv', 'freqs_min', 'freqs_max', 'freqs_mean',
#                'mulliken_min', 'mulliken_max', 'mulliken_atom_0', 'mulliken_atom_1'], axis=1)
#         df3 = dd.read_parquet(f'../data/FE019/raw/{tt}_public_kernel_QM9.parquet')
#         df3 = df3[df3['type'] == 0]
#         df3['type'] = df3['type'].map(type_dict)
#         df3 = df3.drop(['NO','molecule_name'], axis=1)
        
#         df_new = dd.merge(df, df2, on=['id','type'])
#         df_new = dd.merge(df_new, df3, on=['id','type'])
#         df_new.to_parquet(f'../data/FE019/FE019-{tt}-{type_dict[t]}.parquet')
#         break
#     break

In [7]:
print(df.columns.intersection(df2.columns))
print(df.columns.intersection(df3.columns))
print(df2.columns.intersection(df3.columns))

Index(['id', 'type'], dtype='object')
Index(['id', 'type'], dtype='object')
Index(['id', 'type'], dtype='object')


In [16]:
[x for x in df_new.columns if x not in df.columns]

['dist_C_0_x',
 'dist_C_1_x',
 'dist_C_2_x',
 'dist_C_3_x',
 'dist_C_4_x',
 'dist_F_0_x',
 'dist_F_1_x',
 'dist_F_2_x',
 'dist_F_3_x',
 'dist_F_4_x',
 'dist_H_0_x',
 'dist_H_1_x',
 'dist_H_2_x',
 'dist_H_3_x',
 'dist_H_4_x',
 'dist_N_0_x',
 'dist_N_1_x',
 'dist_N_2_x',
 'dist_N_3_x',
 'dist_N_4_x',
 'dist_O_0_x',
 'dist_O_1_x',
 'dist_O_2_x',
 'dist_O_3_x',
 'dist_O_4_x',
 'dist_C_0_y',
 'dist_C_1_y',
 'dist_C_2_y',
 'dist_C_3_y',
 'dist_C_4_y',
 'dist_F_0_y',
 'dist_F_1_y',
 'dist_F_2_y',
 'dist_F_3_y',
 'dist_F_4_y',
 'dist_H_0_y',
 'dist_H_1_y',
 'dist_H_2_y',
 'dist_H_3_y',
 'dist_H_4_y',
 'dist_N_0_y',
 'dist_N_1_y',
 'dist_N_2_y',
 'dist_N_3_y',
 'dist_N_4_y',
 'dist_O_0_y',
 'dist_O_1_y',
 'dist_O_2_y',
 'dist_O_3_y',
 'dist_O_4_y',
 'dist',
 'dist_inv2',
 'distance_closest_0',
 'distance_closest_1',
 'distance_farthest_0',
 'distance_farthest_1',
 'molecule_atom_index_0_dist_mean',
 'molecule_atom_index_0_dist_mean_diff',
 'molecule_atom_index_0_dist_min',
 'molecule_atom_index